In [ ]:
import pandas as pd

In [6]:
arquivo = '../Iniciação Científica/Códigos/microdados_tratado.csv'

microdados = pd.read_csv(arquivo, sep=';', encoding='ISO-8859-1')

dtypes = pd.read_json("dtypes.json", typ='series')
microdados = microdados.astype(dtypes)

microdados = microdados.sample(10000, random_state=42)

microdados = microdados[(microdados['TP_PRESENCA_GERAL'] == 1) & (microdados['NU_MEDIA_GERAL'] != -1)]


microdados.to_csv('sample.csv', sep=';', index=False, encoding='ISO-8859-1')

In [8]:
microdados['NU_MEDIA_GERAL'].unique()

array([498.1 , 542.9 , 445.3 , ..., 402.75, 523.8 , 701.9 ])